# Two Formulations of Maxwell's equations in Cartesian Coordinates

## Author: Zach Etienne
### Formatting improvements courtesy Brandon Clark

### NRPy+ Source Code for this module: 
* [Maxwell/MaxwellCartesian_Evol.py](../edit/Maxwell/MaxwellCartesian_Evol.py)
* [Maxwell/MaxwellCartesian_ID.py](../edit/Maxwell/MaxwellCartesian_ID.py)

## Introduction:

This tutorial will draw on previous work done by Ian Ruchlin on [Maxwell's equations in Curvilinear Coordinates](Tutorial-MaxwellCurvilinear.ipynb), which itself drew on the two formulations described in [Illustrating Stability Properties of Numerical Relativity in Electrodynamics](https://arxiv.org/abs/gr-qc/0201051). This will be done to aid construction of an Einstein Toolkit thorn, which will itself be built in the [Tutorial-ETK_thorn-Maxwell](Tutorial-ETK_thorn-Maxwell.ipynb) tutorial module.

The construction of our equations here will be nearly identical; however, by assuming Cartesian coordinates, we are able to make several simplifications and eliminate the need for [reference_metric.py](../edit/reference_metric.py) as a dependency.

We will begin with their System I. While the Curvilinear version of this code assumed flat spacetime, we will be constructing the equations in a general spacetime. This allows us to simply replace the reference metric "hatted" quantities with their general counterparts.

<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 2](#laplacian): Constructing the Covariant Laplacian 
1. [Step 3](#violation): Measuring the Constarint violation
1. [Step 4](#code_validation_sys1): Code Validation against Maxwell.MaxwellCartesian_Evol NRPy+ Module (System I)
1. [Step 5](#code_validation_sys2): Code Validation against Maxwell.MaxwellCartesian_Evol NRPy+ Module (System II)
1. [Step 6](#id): Constructing the Initial Data
1. [Step 7](#code_validation): Code Validation against Maxwell.MaxwellCartesian_ID  NRPy+ Module
1. [Step 8](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

In [1]:
#Step P1: Import needed Python modules
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

#Step P2: Name this Python module
thismodule = "MaxwellCartesian"

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)

# Step 2: Register gridfunctions that are needed as input.
psi = gri.register_gridfunctions("EVOL", ["psi"])

# Step 3a: Declare the rank-1 indexed expressions E_{i}, A_{i},
#          and \partial_{i} \psi. Derivative variables like these
#          must have an underscore in them, so the finite
#          difference module can parse the variable name properly.
ED = ixp.register_gridfunctions_for_single_rank1("EVOL", "ED")
AD = ixp.register_gridfunctions_for_single_rank1("EVOL", "AD")
psi_dD = ixp.declarerank1("psi_dD")
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])

## Step 3b: Declare the conformal metric tensor and its first 
#           derivative. These are needed to find the Christoffel
#           symbols, which we need for covariant derivatives.
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUX","gammaDD", "sym01") # The AUX or EVOL designation is *not*
                                                                                # used in diagnostic modules.
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01")
gammaDD_dDD = ixp.declarerank4("gammaDD_dDD","sym01_sym23")

gammaUU = ixp.declarerank3("gammaUU","sym01")
detgamma = gri.register_gridfunctions("AUX",["detgamma"])
gammaUU, detgamma = ixp.symm_matrix_inverter3x3(gammaDD)
gammaUU_dD = ixp.declarerank3("gammaDD_dD","sym01")

# Define the Christoffel symbols
GammaUDD = ixp.zerorank3(DIM)
for i in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for m in range(DIM):
                GammaUDD[i][k][l] += (sp.Rational(1,2))*gammaUU[i][m]*\
                                     (gammaDD_dD[m][k][l] + gammaDD_dD[m][l][k] - gammaDD_dD[k][l][m])

# Step 3b: Declare the rank-2 indexed expression \partial_{j} A_{i},
#          which is not symmetric in its indices.
#          Derivative variables like these must have an underscore
#          in them, so the finite difference module can parse the
#          variable name properly.
AD_dD = ixp.declarerank2("AD_dD", "nosym")

# Step 3c: Declare the rank-3 indexed expression \partial_{jk} A_{i},
#          which is symmetric in the two {jk} indices.
AD_dDD = ixp.declarerank3("AD_dDD", "sym12")

# Step 4: Calculate first and second covariant derivatives, and the
#         necessary contractions.
# First covariant derivative
# D_{j} A_{i} = A_{i,j} - \Gamma^{k}_{ij} A_{k}
AD_dcovD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        AD_dcovD[i][j] = AD_dD[i][j]
        for k in range(DIM):
            AD_dcovD[i][j] -= GammaUDD[k][i][j] * AD[k]


<a id='laplacian'></a>

# Step 2: Constructing the Covariant Laplacian \[Back to [top](#toc)\]
$$\label{laplacian}$$

One particular difficulty we will encounter here is taking the covariant Laplacian of a vector. This will here take the form of $D_j D^j A_i$. We will start with the outer derivative after lowering the index on the second operator. So, we see that 
\begin{align}
D_j D^j A_i &= D_j (\gamma^{jk} D_k A_i) \\
&= (D_k A_i) D_j \gamma^{jk} + \gamma^{jk} D_j D_k A_i \\
&= \gamma^{jk} [\partial_j (D_k A_i) - \Gamma^l_{ij} D_k A_l - \Gamma^l_{jk} D_l A_i],
\end{align}
dropping the first term from the second line because $D_j \gamma^{jk} = 0$ Next, we will again apply the covariant derivative to $A_i$. First, however, we should consider that 
\begin{align}
D_k A_i &= \partial_k A_i - \Gamma^l_{ik} A_l \\
& = \partial_k A_i - \gamma^{lm} \Gamma_{mik} A_l \\
& = \partial_k A_i - \Gamma_{mik} A^m, \\
\end{align}
where $\Gamma_{ljk} = \frac{1}{2} (\partial_k \gamma_{lj} + \partial_j \gamma_{kl} - \partial_l \gamma_{jk})$ is the Christoffel symbol of the first kind. Note how we were able to use the raising operator to switch the height of two indices; we will use this in upcoming steps. Thus, our expression becomes
\begin{align}
D_j D^j A_i &= \gamma^{jk} [\partial_j \partial_k A_i - \partial_j (\Gamma_{lik} A^l) - \Gamma^l_{ij} \partial_k A_l + \Gamma^m_{ij} \Gamma_{lmk} A^l - \Gamma^l_{jk} \partial_l A_i + \Gamma^m_{jk} \Gamma_{lim} A^l] \\
&= \gamma^{jk} [\partial_j \partial_k A_i - \underbrace{\partial_j (\Gamma_{lik} A^l)}_{\text{sub-term}} - \Gamma^l_{ij} \partial_k A_l + \Gamma^m_{ij} \Gamma^l_{mk} A_l - \Gamma^l_{jk} \partial_l A_i + \Gamma^m_{jk} \Gamma^l_{im} A_l].
\end{align}
Let's focus on the underbraced sub-term for a moment. Expanding this using the product rule and the definition of the Christoffel symbol, 
\begin{align}
\partial_j (\Gamma_{lik} A^l) &= A^l \partial_j \Gamma_{lik} + \Gamma_{lik} \partial_j A^l \\
&= A^l \partial_j (\partial_k \gamma_{li} + \partial_i \gamma_{kl} - \partial_l \gamma_{ik}) + \Gamma_{lik} \partial_j (\gamma^{lm} A_m) \\
&= A^l (\gamma_{li,kj} + \gamma_{kl,ij} - \gamma_{ik,lj}) + \Gamma_{lik} (\gamma^{lm} A_{m,j} + A_m \gamma^{lm}{}_{,j}), \\
\end{align}
where commas in subscripts denote partial derivatives.

So, the Laplacian becomes 
\begin{align}
D_j D^j A_i &= \gamma^{jk} [A_{i,jk} - 
\underbrace{A^l (\gamma_{li,kj} + \gamma_{kl,ij} - \gamma_{ik,lj})}_{\text{Term 1}} + 
\underbrace{\Gamma_{lik} (\gamma^{lm} A_{m,j} + A_m \gamma^{lm}{}_{,j})}_{\text{Term 2}} - 
\underbrace{(\Gamma^l_{ij} A_{l,k} + \Gamma^l_{jk} A_{i,l})}_{\text{Term 3}} + 
\underbrace{(\Gamma^m_{ij} \Gamma^l_{mk} A_l + \Gamma ^m_{jk} \Gamma^l_{im} A_l)}_{\text{Term 4}}]; \\
\end{align}
we will now begin to contruct these terms individually.

In [2]:
# First, we must construct the lowered Christoffel symbols:
# \Gamma_{ijk} = \gamma_{il} \Gamma^l_{jk}
# And raise the index on A:
# A^j = \gamma^{ij} A_i
GammaDDD = ixp.zerorank3()
AU = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        AU[j] = gammaUU[i][j] * AD[i]
        for k in range(DIM):
            for l in range(DIM):
                GammaDDD[i][j][k] = gammaDD[i][l] * GammaUDD[l][j][k]

# Covariant second derivative (the bracketed terms):
# D_j D^j A_i = \gamma^{jk} [A_{i,jk} - A^l (\gamma_{li,kj} + \gamma_{kl,ij} - \gamma_{ik,lj})
#               + \Gamma_{lik} (\gamma^{lm} A_{m,j} + A_m \gamma^{lm}{}_{,j})
#               - (\Gamma^l_{ij} A_{l,k} + \Gamma^l_{jk} A_{i,l})
#               + (\Gamma^m_{ij} \Gamma^l_{mk} A_l + \Gamma ^m_{jk} \Gamma^l_{im} A_l)
AD_dcovDD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            AD_dcovDD[i][j][k] = AD_dDD[i][j][k]
            for l in range(DIM):
                # Terms 1 and 3
                AD_dcovDD[i][j][k] -= AU[l] * (gammaDD_dDD[l][i][k][j] + gammaDD_dDD[k][l][i][j] - \
                                               gammaDD_dDD[i][k][l][j]) \
                                    + GammaUDD[l][i][j] * AD_dD[l][k] + GammaUDD[l][j][k] * AD_dD[i][l]
                for m in range(DIM):
                    # Terms 2 and 4
                    AD_dcovDD[i][j][k] += GammaDDD[l][i][k] * (gammaUU[l][m] * AD_dD[m][j] + AD[m] * gammaUU_dD[l][m][j]) \
                                        + GammaUDD[m][i][j] * GammaUDD[l][m][k] * AD[l] \
                                        + GammaUDD[m][j][k] * GammaUDD[l][i][m] * AD[l]

# Covariant divergence
# D_{i} A^{i} = \gamma^{ij} D_{j} A_{i}
DivA = 0
# Gradient of covariant divergence
# DivA_dD_{i} = \gamma^{jk} A_{k;\hat{j}\hat{i}}
DivA_dD = ixp.zerorank1()
# Covariant Laplacian
# LapAD_{i} = \gamma^{jk} A_{i;\hat{j}\hat{k}}
LapAD = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        DivA += gammaUU[i][j] * AD_dcovD[i][j]
        for k in range(DIM):
            DivA_dD[i] += gammaUU[j][k] * AD_dcovDD[k][j][i]
            LapAD[i]   += gammaUU[j][k] * AD_dcovDD[i][j][k]

# Step 5: Define right-hand sides for the evolution.
ArhsD = ixp.zerorank1()
ErhsD = ixp.zerorank1()
for i in range(DIM):
    ArhsD[i] = -ED[i] - psi_dD[i]
    ErhsD[i] = -LapAD[i] + DivA_dD[i]
psi_rhs = -DivA

# Step 6: Generate C code for System I Maxwell's evolution equations,
#         print output to the screen (standard out, or stdout).
#lhrh_list = []
#for i in range(DIM):
#    lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "AD" + str(i)), rhs=ArhsD[i]))
#    lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "ED" + str(i)), rhs=ErhsD[i]))
#lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "psi"), rhs=psi_rhs))
    
#fin.FD_outputC("stdout", lhrh_list)

<a id='violation'></a>

# Step 3: Measuring the Constarint violation \[Back to [top](#toc)\]
$$\label{violation}$$

To evaluate our results, we will need to measure how much our simulation differs from what should be physically possible. We will do this with the constraint equation $D_i E^i = 4 \pi \rho_e$; specifically, we will measure the constraint violation 
\begin{align}
\mathcal{C} &= D_i E^i - 4 \pi \rho_e \\
&= D_i (\gamma^{ij} E_j) - 4 \pi \rho_e \\
&= \gamma^{ij} D_i E_j + E_j D_i \gamma^{ij} - 4 \pi \rho_e \\
&= \gamma^{ij} D_i E_j,
\end{align}
since the covariant derivative of the metric tensor is $0$ and $\rho_e=0$ in free space. So, $\mathcal{C} = \gamma^{ij} (E_{j,i} - \Gamma^b_{ij} E_b)$, which will be valid for both systems.

In [3]:
ED_dD = ixp.declarerank2("ED_dD","nosym")
Cviola = gri.register_gridfunctions("AUX", ["Cviola"])
Cviola = sp.sympify(0)
for i in range(DIM):
    for j in range(DIM):
        Cviola = gammaUU[i][j] * ED_dD[j][i] 
        for b in range(DIM):
            Cviola -= GammaUDD[b][i][j] * ED[b]


<a id='code_validation_sys1'></a>

# Step 4: Code Validation against Maxwell.MaxwellCartesian_Evol NRPy+ Module (System I) \[Back to [top](#toc)\]
$$\label{code_validation_sys1}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the RHSs of Maxwell's equations (in System I) between

1. this tutorial and 
2. the NRPy+ [Maxwell.MaxwellCartesian](../edit/Maxwell/MaxwellCartesian_Evol.py) module.


In [4]:
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []

# Step 18: Call the MaxwellCartesian_Evol() function from within the
#          Maxwell/MaxwellCartesian_Evol.py module,
#          which should do exactly the same as in Steps 1-16 above.
print("vvv Ignore the minor warning below. vvv")

import Maxwell.MaxwellCartesian_Evol as mwevol
par.set_parval_from_str("System_to_use","System_I")
mwevol.MaxwellCartesian_Evol()

print("^^^ Ignore the minor warning above. ^^^\n")

print("Consistency check between MaxwellCartesian tutorial and NRPy+ module: ALL SHOULD BE ZERO.")

print("psi_rhs - mwevol.psi_rhs = " + str(psi_rhs - mwevol.psi_rhs))
for i in range(DIM):

    print("ArhsD["+str(i)+"] - mwevol.ArhsD["+str(i)+"] = " + str(ArhsD[i] - mwevol.ArhsD[i]))
    print("ErhsD["+str(i)+"] - mwevol.ErhsD["+str(i)+"] = " + str(ErhsD[i] - mwevol.ErhsD[i]))

print("Cviola - mwevol.Cviola = " + str(Cviola - mwevol.Cviola))


vvv Ignore the minor warning below. vvv
^^^ Ignore the minor warning above. ^^^

Consistency check between MaxwellCartesian tutorial and NRPy+ module: ALL SHOULD BE ZERO.
psi_rhs - mwevol.psi_rhs = 0
ArhsD[0] - mwevol.ArhsD[0] = 0
ErhsD[0] - mwevol.ErhsD[0] = 0
ArhsD[1] - mwevol.ArhsD[1] = 0
ErhsD[1] - mwevol.ErhsD[1] = 0
ArhsD[2] - mwevol.ArhsD[2] = 0
ErhsD[2] - mwevol.ErhsD[2] = 0
Cviola - mwevol.Cviola = -ED0*(gammaDD_dD222*(gammaDD01*gammaDD12 - gammaDD02*gammaDD11)/(2*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)) + (2*gammaDD_dD022 - gammaDD_dD220)*(gammaDD11*gammaDD22 - gammaDD12**2)/(2*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)) + (2*gammaDD_dD122 - gammaDD_dD221)*(-gammaDD01*gammaDD22 + gammaDD02*gammaDD12)/(2*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD

We will now build the equations for System II.

In [5]:
# We inherit here all of the definitions from System I, above

# Step 7a: Register the scalar auxiliary variable \Gamma
Gamma = gri.register_gridfunctions("EVOL", ["Gamma"])

# Step 7b: Declare the ordinary gradient \partial_{i} \Gamma
Gamma_dD = ixp.declarerank1("Gamma_dD")

# Step 8a: Construct the second covariant derivative of the scalar \psi
# \psi_{;\hat{i}\hat{j}} = \psi_{,i;\hat{j}}
#                        = \psi_{,ij} - \Gamma^{k}_{ij} \psi_{,k}
psi_dDD = ixp.declarerank2("psi_dDD", "sym01")
psi_dcovDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        psi_dcovDD[i][j] = psi_dDD[i][j]
        for k in range(DIM):
            psi_dcovDD[i][j] += - GammaUDD[k][i][j] * psi_dD[k]

# Step 8b: Construct the covariant Laplacian of \psi
# Lappsi = ghat^{ij} D_{j} D_{i} \psi
Lappsi = 0
for i in range(DIM):
    for j in range(DIM):
        Lappsi += gammaUU[i][j] * psi_dcovDD[i][j]

# Step 9: Define right-hand sides for the evolution.
ArhsD = ixp.zerorank1()
ErhsD = ixp.zerorank1()
for i in range(DIM):
    ArhsD[i] = -ED[i] - psi_dD[i]
    ErhsD[i] = -LapAD[i] + Gamma_dD[i]
psi_rhs = -Gamma
Gamma_rhs = -Lappsi
    
# Step 10: Generate C code for System II Maxwell's evolution equations,
#          print output to the screen (standard out, or stdout).
#lhrh_list = []
#for i in range(DIM):
#    lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "AD" + str(i)), rhs=ArhsD[i]))
#    lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "ED" + str(i)), rhs=ErhsD[i]))
#lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "psi"), rhs=psi_rhs))
#lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "Gamma"), rhs=Gamma_rhs))
    
#fin.FD_outputC("stdout", lhrh_list)

<a id='code_validation_sys2'></a>

# Step 5: Code Validation against Maxwell.MaxwellCartesian_Evol NRPy+ Module (System II) \[Back to [top](#toc)\]
$$\label{code_validation_sys2}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the RHSs of Maxwell's equations (in System II) between

1. this tutorial and 
2. the NRPy+ [Maxwell.MaxwellCartesian](../edit/Maxwell/MaxwellCartesian_Evol.py) module.


In [6]:
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []

# Step 18: Call the MaxwellCartesian_Evol() function from within the
#          Maxwell/MaxwellCartesian_Evol.py module,
#          which should do exactly the same as in Steps 1-16 above.

par.set_parval_from_str("System_to_use","System_II")
mwevol.MaxwellCartesian_Evol()

print("Consistency check between MaxwellCartesian tutorial and NRPy+ module: ALL SHOULD BE ZERO.")

print("psi_rhs - mwevol.psi_rhs = " + str(psi_rhs - mwevol.psi_rhs))
print("Gamma_rhs - mwevol.Gamma_rhs = " + str(Gamma_rhs - mwevol.Gamma_rhs))
for i in range(DIM):

    print("ArhsD["+str(i)+"] - mwevol.ArhsD["+str(i)+"] = " + str(ArhsD[i] - mwevol.ArhsD[i]))
    print("ErhsD["+str(i)+"] - mwevol.ErhsD["+str(i)+"] = " + str(ErhsD[i] - mwevol.ErhsD[i]))

print("Cviola - mwevol.Cviola = " + str(Cviola - mwevol.Cviola))


Consistency check between MaxwellCartesian tutorial and NRPy+ module: ALL SHOULD BE ZERO.
psi_rhs - mwevol.psi_rhs = 0
Gamma_rhs - mwevol.Gamma_rhs = 0
ArhsD[0] - mwevol.ArhsD[0] = 0
ErhsD[0] - mwevol.ErhsD[0] = 0
ArhsD[1] - mwevol.ArhsD[1] = 0
ErhsD[1] - mwevol.ErhsD[1] = 0
ArhsD[2] - mwevol.ArhsD[2] = 0
ErhsD[2] - mwevol.ErhsD[2] = 0
Cviola - mwevol.Cviola = -ED0*(gammaDD_dD222*(gammaDD01*gammaDD12 - gammaDD02*gammaDD11)/(2*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)) + (2*gammaDD_dD022 - gammaDD_dD220)*(gammaDD11*gammaDD22 - gammaDD12**2)/(2*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)) + (2*gammaDD_dD122 - gammaDD_dD221)*(-gammaDD01*gammaDD22 + gammaDD02*gammaDD12)/(2*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02

<a id='id'></a>

# Step 6: Constructing the Initial Data \[Back to [top](#toc)\]
$$\label{id}$$

Now that we have evolution equations in place, we must construct the initial data that will be evolved by the solver of our choice. We will start from the analytic solution to this system of equations, given in [Illustrating Stability Properties of Numerical Relativity in Electrodynamics](https://arxiv.org/abs/gr-qc/0201051) as
\begin{align}
A^{\hat{\phi}} &= \mathcal{A} \sin \theta \left( \frac{e^{-\lambda v^2}-e^{-\lambda u^2}}{r^2} - 2 \lambda \frac{ve^{-\lambda v^2}-ue^{-\lambda u^2}}{r} \right), \\
\end{align}
for vanishing scalar potential $\psi$, where $\mathcal{A}$ gives the amplitude, $\lambda$ describes the size of the wavepacket, $u = t+r$, and $v = t-r$. Other components of this field are $0$.To get initial data, then, we simply set $t=0$; since $\psi=0$, $E_i = \partial_t A_i$. Thus, our initial data becomes the equations
\begin{align}
A^{\hat{\phi}} &= 0 \\
E^{\hat{\phi}} &= 8 \mathcal{A} r \sin \theta \lambda^2 e^{-\lambda r^2} \\
\psi &= 0
\end{align}
where the non-$\hat{\phi}$ components are set to 0. We still will need to convert $E^i$ in spherical-like coordinates and then lower its index. Using the standard transformations for coordinates and unit vectors, 
\begin{align}
E^{\hat{x}} &= -\frac{y E^{\hat{\phi}}(x,y,z)}{\sqrt{x^2+y^2}} \\
E^{\hat{y}} &= \frac{x E^{\hat{\phi}}(x,y,z)}{\sqrt{x^2+y^2}} \\
E^{\hat{z}} &= 0. \\
\end{align}
We can lower the index in the usual way.

For system II, we will also need to set initial data for $\Gamma$. Since $\Gamma = -\partial_t \psi$ and we have chosen $\psi(t=0) = 0$, $\Gamma(t=0) = 0$. 

In [7]:
# Step 1: Declare free parameters intrinsic to these initial data
amp = par.Cparameters("REAL",thismodule,"amp")
lam = par.Cparameters("REAL",thismodule,"lam")

# Step 2: Set the initial data
AidD = ixp.zerorank1()

EidD = ixp.zerorank1()
EidU = ixp.zerorank1()
# Set the coordinate transformations:
radial = sp.sqrt(x*x + y*y + z*z)
polar = sp.atan2(sp.sqrt(x*x + y*y),z)
EU_phi = 8*amp*radial*sp.sin(polar)*lam*lam*sp.exp(-lam*radial*radial)
EidU[0] = -(y * EU_phi)/sp.sqrt(x*x + y*y)
EidU[1] = (x * EU_phi)/sp.sqrt(x*x + y*y)
# The z component (2)is zero. 
for i in range(DIM):
    for j in range(DIM):
        EidD[i] += gammaDD[i][j] * EidU[j]
      
psi_ID = sp.sympify(0)
Gamma_ID = sp.sympify(0)

<a id='code_validation'></a>

# Step 7: Code Validation against Maxwell.MaxwellCartesian_ID  NRPy+ Module \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the initial data we intend to use between

1. this tutorial and 
2. the NRPy+ [Maxwell.MaxwellCartesian_ID](../edit/Maxwell/MaxwellCartesian_ID.py) module.

Since the initial data is identical between the two systems for $E_i$, $A_i$, and $\psi$, so checking system I should be redundant; we will do it anyways, to be sure.

In [8]:
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []
par.set_parval_from_str("System_to_use","System_I")

print("vvv Ignore the minor warning below. vvv")

import Maxwell.MaxwellCartesian_ID as mwid
mwid.MaxwellCartesian_ID()

print("^^^ Ignore the minor warning above. ^^^\n")

print("Consistency check between MaxwellCartesian tutorial and NRPy+ module: ALL SHOULD BE ZERO.")

print("psi_ID - mwid.psi_ID = " + str(psi_ID - mwid.psi_ID))
for i in range(DIM):

    print("AidD["+str(i)+"] - mwid.AidD["+str(i)+"] = " + str(AidD[i] - mwid.AidD[i]))
    print("EidD["+str(i)+"] - mwid.EidD["+str(i)+"] = " + str(EidD[i] - mwid.EidD[i]))


vvv Ignore the minor warning below. vvv
^^^ Ignore the minor warning above. ^^^

Consistency check between MaxwellCartesian tutorial and NRPy+ module: ALL SHOULD BE ZERO.
psi_ID - mwid.psi_ID = 0
AidD[0] - mwid.AidD[0] = 0
EidD[0] - mwid.EidD[0] = 0
AidD[1] - mwid.AidD[1] = 0
EidD[1] - mwid.EidD[1] = 0
AidD[2] - mwid.AidD[2] = 0
EidD[2] - mwid.EidD[2] = 0


Finally, we will repeat the check with system II initial data.

In [9]:
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []
par.set_parval_from_str("System_to_use","System_II")

print("vvv Ignore the minor warning below. vvv")

mwid.MaxwellCartesian_ID()

print("^^^ Ignore the minor warning above. ^^^\n")

print("Consistency check between MaxwellCartesian tutorial and NRPy+ module: ALL SHOULD BE ZERO.")

print("psi_ID - mwid.psi_ID = " + str(psi_ID - mwid.psi_ID))
print("Gamma_ID - mwid.Gamma_ID = " + str(Gamma_ID - mwid.Gamma_ID))
for i in range(DIM):

    print("AidD["+str(i)+"] - mwid.AidD["+str(i)+"] = " + str(AidD[i] - mwid.AidD[i]))
    print("EidD["+str(i)+"] - mwid.EidD["+str(i)+"] = " + str(EidD[i] - mwid.EidD[i]))


vvv Ignore the minor warning below. vvv
initialize_param() minor warning: Did nothing; already initialized parameter Maxwell.MaxwellCartesian_ID::amp
initialize_param() minor warning: Did nothing; already initialized parameter Maxwell.MaxwellCartesian_ID::lam
^^^ Ignore the minor warning above. ^^^

Consistency check between MaxwellCartesian tutorial and NRPy+ module: ALL SHOULD BE ZERO.
psi_ID - mwid.psi_ID = 0
Gamma_ID - mwid.Gamma_ID = 0
AidD[0] - mwid.AidD[0] = 0
EidD[0] - mwid.EidD[0] = 0
AidD[1] - mwid.AidD[1] = 0
EidD[1] - mwid.EidD[1] = 0
AidD[2] - mwid.AidD[2] = 0
EidD[2] - mwid.EidD[2] = 0


<a id='latex_pdf_output'></a>

# Step 8: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-MaxwellCartesian.pdf](Tutorial-MaxwellCartesian.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [10]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-MaxwellCartesian.ipynb
!pdflatex -interaction=batchmode Tutorial-MaxwellCartesian.tex
!pdflatex -interaction=batchmode Tutorial-MaxwellCartesian.tex
!pdflatex -interaction=batchmode Tutorial-MaxwellCartesian.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-MaxwellCartesian.ipynb to latex
[NbConvertApp] Writing 126142 bytes to Tutorial-MaxwellCartesian.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
